In [57]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)

# Load MNIST dataset
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()
(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

In [58]:
# # Downsample the images by factor 2
# ratio = 1/2

# Xtrain_down = np.ones((ntrain, xdim*ratio, ydim*ratio))
# for i in range(ntrain):
#     Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], downsample_ratio)

# Xtest_down = np.ones((ntest, xdim*ratio, ydim*ratio))
# for i in range(ntest):
#     Xtest_down[i,:,:] = imresize(Xtest[i,:,:], downsample_ratio)

# plt.imshow(Xtrain[1,:,:])
# plt.imshow(Xtrain_down[1,:,:])

In [59]:
# vectorize the input images
# Xtrain_down = Xtrain_down.reshape(ntrain, xdim*ratio**2).astype('float32') / 255
# Xtest_down = Xtest_down.reshape(ntest, xdim*ratio**2).astype('float32') / 255
Xtrain = Xtrain.reshape(ntrain, xdim**2).astype('float32') / 255
Xtest = Xtest.reshape(ntest, xdim**2).astype('float32') / 255
Ytrain = np_utils.to_categorical(Ytrain, 10)
Ytest = np_utils.to_categorical(Ytest, 10)

print(Xtrain.shape)
print(Ytrain.shape)

(60000, 784)
(60000, 10)


In [60]:
##### TRAIN A FULLY-CONNECTED NN WITH TWO HIDDEN LAYERS

# create model
model = Sequential()
model.add(Dense(100, input_shape=(xdim**2,), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(Xtrain, Ytrain, nb_epoch=25, batch_size=16)

# evaluate the model
scores = model.evaluate(Xtest, Ytest, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# # calculate predictions
# predictions = model.predict(Xtest)
# # round predictions
# rounded = [round(x[0]) for x in predictions]
# print(rounded)

/Users/elahe/anaconda3/lib/python3.5/site-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/25
60000/60000 [==============================] - 18s - loss: 0.2290 - acc: 0.9314    
Epoch 2/25
60000/60000 [==============================] - 25s - loss: 0.1054 - acc: 0.9675    
Epoch 3/25
60000/60000 [==============================] - 21s - loss: 0.0777 - acc: 0.9764    
Epoch 4/25
60000/60000 [==============================] - 20s - loss: 0.0619 - acc: 0.9807    
Epoch 5/25
60000/60000 [==============================] - 24s - loss: 0.0519 - acc: 0.9830    
Epoch 6/25
60000/60000 [==============================] - 35s - loss: 0.0432 - acc: 0.9865    
Epoch 7/25
60000/60000 [==============================] - 23s - loss: 0.0372 - acc: 0.9884    
Epoch 8/25
60000/60000 [==============================] - 19s - loss: 0.0344 - acc: 0.9893    
Epoch 9/25
60000/60000 [==============================] - 19s - loss: 0.0318 - acc: 0.9898    
Epoch 10/25
60000/60000 [==============================] - 17s - loss: 0.0274 - acc: 0.9915    
Epoch 11/25
60000/60000 [========================

In [61]:
# from PIL import Image
# from resizeimage import resizeimage

# img = Xtrain[1,:,:]
# new_img = resizeimage.resize_width(img, 7)
# plt.imshow(new_img)
